### 1.  Problem Statement:
- The problem states that 30% of the candidates who accept the job offer do not join the company. HRWorks wants to find if a model can be built to predict the likelihood of a candidate joining the company.<br>
- Now we have to compare all the factors that tell us that what factors are affecting for a candidate so that they are not joining the company.<br>
- We have to also relate these factors to understand whether a candidate will join or not based on the given circumstances in future.

###  Why it is needed:

- As we know taking interviews and exams of applied candidates includes some kind of resources and time which may lead to loss if a candidate doesn't join after clearing all the stages.<br>
- Because of this sometime we loose a candidate who is willing to work but scored less in eligibility criteria.<br>
- Sometimes we loose a good or best candidate or we can say desired candidate that will help to increase the future of the company ot will increase the revenue of the company in some way. So we want to understand what the problem they are facing in joing the company.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
import sklearn
import statsmodels.api as sm 
import scipy.stats as st
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
sns.set()

In [ ]:
data=pd.read_csv("../input/hr-works/hr_data.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.describe().T

As we can see that there are outliers present in Are which are needed in the analysis

In [ ]:
data.describe(include=object).T

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.dropna()

## Treating missing values

As age contains outliers we will use <b> Median </b> instead of <b> Mean </b>

In [ ]:
data['Age'].fillna(data["Age"].median(),inplace=True)

In [ ]:
data['Offered band'].value_counts()

In [ ]:
imputer=data.copy()

In [ ]:
freq_imputer=SimpleImputer(strategy='most_frequent')

In [ ]:
imputer.loc[:,['Offered band']]=freq_imputer.fit_transform(imputer.loc[:,['Offered band']])

In [ ]:
data=imputer.copy()

In [ ]:
data.isnull().sum()

### 2. Hypothesis:

- H<sub>0</sub>: Based on the given information by a candidate will join
- H<sub>1</sub>: Based on the given information by a candidate will not join

### 3. Data Exploration

In [ ]:
data.columns

In [ ]:
data['Status'].value_counts()

In [ ]:
sns.countplot(data['Status'])
plt.xticks(rotation=90)
plt.xlabel('These are Job Status');

Most of the candidate have joined

In [ ]:
def countp(x, data):
    plt.figure(figsize = (10,8))
    sns.countplot(x = x, data = data, hue = 'Status', palette = 'rocket')
    plt.xlabel(x)
    plt.xticks(rotation=90)

In [ ]:
countp('Age',data)

We can see that people falling under 34 years age are most joined and not joined

In [ ]:
countp('Gender',data)

Males are most joined and not joined among genders

In [ ]:
countp('DOJ Extended',data)

Those who got Date of Joining, extended joined and not joined slightly more 

In [ ]:
countp('Notice period',data)

Those who got a notice period of 30 days joined and not joined more

In [ ]:
countp('Offered band',data)

Most of the people joined and not joined E1 Offered band

In [ ]:
countp('Location',data)

Most of the people joined are from Chennai and Noida while not joined are Chennai and Banglore

In [ ]:
countp('LOB',data)

MOst of the people joined in INFRA and ERS Line of Bussiness while not joined in ERS, INFRA and BFSi

In [ ]:
countp('Joining Bonus',data)

Those who didn't got any joining bonus are most who joined or not joined

In [ ]:
countp('Rex in Yrs',data)

Those who got age relaxation between 3 to 4 yrs joined and not joined more

In [ ]:
countp('Candidate Source',data)

Those who came direct are the most joined and not joined while those who are refered by company are least joined and not joined

### Treatment of outliers

Removing Outliers from Age

In [ ]:
sns.distplot(data['Age']);

In [ ]:
data=data[data['Age'] < data['Age'].quantile(.99)]

In [ ]:
sns.distplot(data['Age']);

Removing Outliers from Pecent hike expected in CT

In [ ]:
sns.distplot(data['Pecent hike expected in CTC']);

In [ ]:
data=data[data['Pecent hike expected in CTC'] < data['Pecent hike expected in CTC'].quantile(.99)]

In [ ]:
sns.distplot(data['Pecent hike expected in CTC']);

Removing Outliers from Percent hike offered in CTC

In [ ]:
sns.distplot(data['Percent hike offered in CTC']);

In [ ]:
data=data[data['Percent hike offered in CTC'] < data['Percent hike offered in CTC'].quantile(.99)]

In [ ]:
sns.distplot(data['Percent hike offered in CTC']);

Removing Outliers from Percent difference CTC

In [ ]:
sns.distplot(data['Percent difference CTC']);

In [ ]:
data=data[data['Percent difference CTC'] < data['Percent difference CTC'].quantile(.99)]

In [ ]:
sns.distplot(data['Percent difference CTC']);

Removing Outliers from Rex in Yrs

In [ ]:
sns.distplot(data['Rex in Yrs']);

In [ ]:
data=data[data['Rex in Yrs'] < data['Rex in Yrs'].quantile(.99)]

In [ ]:
sns.distplot(data['Rex in Yrs']);

## 4. Model

### Logistic Regression

In [ ]:
data['Status']=data['Status'].apply(lambda x:0 if x=='Not Joined' else 1)

In [ ]:
x_feature = list(data.columns)
x_feature.remove('Status') 
x_feature.remove("Pecent hike expected in CTC") 
x_feature.remove("Percent hike offered in CTC")
x_feature.remove("SLNO")
x_feature.remove("Candidate Ref")

In [ ]:
encoded_data = pd.get_dummies(data[x_feature],drop_first=True)

In [ ]:
y = data['Status']
x1 = encoded_data
x = sm.add_constant(x1)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, train_size = 0.8, random_state = 42)

In [ ]:
logit = sm.Logit(y_train, x_train)

In [ ]:
logit=logit.fit(method='bfgs')

In [ ]:
logit.summary2()

In [ ]:
y_pred = logit.predict(x_test)
y_pred

In [ ]:
def get_significant_variables(lm):
    var_p_vals = pd.DataFrame(lm.pvalues)
    var_p_vals['vars'] = var_p_vals.index
    var_p_vals.columns = ['pvals','vars']
    return list(var_p_vals[var_p_vals.pvals <= 0.05]['vars'])

In [ ]:
significant_variables = get_significant_variables(logit)
significant_variables

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(encoded_data[significant_variables].corr(), annot = True, fmt ='.1f')

In [ ]:
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=100)

In [ ]:
log_reg.fit(x_train, y_train)

In [ ]:
y_pred = log_reg.predict(x_test)
y_pred[0:100]

In [ ]:
log_reg.predict_proba(x_test)[0:28]

In [ ]:
log_reg.coef_[0:100]

In [ ]:
def confusion_matrix(actuals,predicted):
    cm = metrics.confusion_matrix(actuals,predicted,[1,0])
    sns.heatmap(cm, annot = True, fmt ='.2f', xticklabels=['Joined', 'Not Joined'], 
                yticklabels=['Joined', 'Not Joined'])
    
    plt.ylabel("Actual Labels")
    plt.xlabel("Predicted Labels")

In [ ]:
print(metrics.classification_report(y_test, log_reg.predict(x_test)))

In [ ]:
confusion_matrix(y_test, y_pred)

### Decision Tree Classifier

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(x_train,y_train)

In [ ]:
y_pred_dt = dtc.predict(x_test)

In [ ]:
y_pred_dt_train = dtc.predict(x_train)

In [ ]:
print(metrics.classification_report(y_test, y_pred_dt))

In [ ]:
confusion_matrix(y_test, y_pred_dt)

### KNeighbors Classifier

In [ ]:
knn_classifier = KNeighborsClassifier()

In [ ]:
knn_classifier.fit(x_train,y_train)

In [ ]:
y_pred_knn=knn_classifier.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred_knn)

### Random Forest Regressor

In [ ]:
forest = RandomForestRegressor(n_estimators=10, random_state=17)
forest.fit(x_train, y_train)

In [ ]:
y_train_pred = forest.predict(x_train)
y_test_pred = forest.predict(x_test)
print('Mean squared error (train): {:.3f}'.format(mean_squared_error(y_train, y_train_pred)))
print('Mean squared error (holdout): {:.3f}'.format(mean_squared_error(y_test, y_test_pred)))

In [ ]:
forest_params = {'max_depth': range(10, 25), 'max_features': range(6,12)}
locally_best_forest = GridSearchCV(forest, forest_params,scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
locally_best_forest.fit(x_train, y_train)
print(abs(locally_best_forest.best_score_), locally_best_forest.best_params_)

In [ ]:
importances = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_,index=x.columns,columns=['Importance'])
importances.sort_values('Importance').plot(kind='barh', figsize = (10,8));

#### 4.Comparing different models.
Out of Logistic Regression(Accuracy= 0.82, True positive values= 1193), Decision Tree Classifier(Accuracy= 0.74, True positive values=1251),), KNeighbors Classifier(Accuracy= 0.82, True positive values=1337), Logistic Regression and KNeighbors Classifier have same accuracy but KNeighbors Classifier have more true predicted values. There out of three KNeighbors Classifier model is best suited for this problem

In [ ]:
pd.DataFrame({'Logistic Regression' :  [0.82, 1193], 'Decision Tree' :  [0.73, 1251], 'KNN Classifier' : [0.82, 1337]}, index=['Accuracy','True positive values'])